## CESM2 - LARGE ENSEMBLE (LENS2)

- The purpose of this notebook is to plot Hovmoller diagrams from vertical profiles. 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.offsetbox import AnchoredText

In [ ]:
# Potential VortiicIy 
#ds_PV = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/PV/PV_south_atl_regions.nc')
#ds_PV.coords['z_t']=ds_PV.coords['z_t']*-0.01
#ds_PV=ds_PV*1000000 # 1/s/cm to 1/s/km

# Potential Density
ds_PD = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/PD/PD_south_atl_regions.nc')
ds_PD.coords['z_t']=ds_PD.coords['z_t']*0.00001
ds_PD=ds_PD*1000 # g/cm3 to kg/m3

# Temperature /glade/scratch/mauricio/Data/LENS2/TEMP
ds_TEMP = xr.open_dataset("/glade/scratch/mauricio/Data/LENS2/TEMP/TEMP_south_atl_regions.nc")
ds_TEMP.coords['z_t']=ds_TEMP.coords['z_t']*0.00001

# Salinity
ds_SALT = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/SALT/SALT_south_atl_regions.nc')
ds_SALT.coords['z_t']=ds_SALT.coords['z_t']*0.00001

# SHF
#ds_SHF = xr.open_dataset('/glade/scratch/mauricio/Data/LENS2/SHF/SHF_south_atl_regions.nc')

In [ ]:
# Set variables, units, etc
var1=('SALT'); label1=(r'$\Delta\,\rm{Salinity}\,[\rm{g\,kg^{-1}}]$'); color1=('BrBG')
var2=('TEMP'); label2=(r'$\Delta\,\rm{Temperature}\,[\rm{^oC}]$'); color2=('PiYG')
var3=('PD'); label3=(r'$\Delta\,\rm{Density}\,[\rm{kg\,m^{-3}}]$'); color3=('PuOr')
#var=('PV'); label=('$\Delta$(1/s/cm)'); color=('PiYG')

In [ ]:
def split_depth_plot(da, 
                     ax_top, 
                     ax_bottom, 
                     depth,
                     mx,
                     mn,
                     cor):
    # Make sure the colour range is the same for the top and bottom plots
    
    orig_map=plt.cm.get_cmap(cor)
    scale_cor=orig_map.reversed()
    N = 50
    
    # Plot the top half
    da.sel(z_t=slice(0,depth)).plot.contourf('time','z_t', vmin=mn, vmax=mx,
                                                add_colorbar=False, 
                                                levels=np.linspace(mn, mx, N),
                                                ax=ax_top, cmap=scale_cor, add_labels=False) 
    CS=da.sel(z_t=slice(0,depth)).plot.contour('time','z_t', vmin=mn, vmax=mx,
                                            add_colorbar=False, 
                                            levels = [0],
                                            ax=ax_top, #colors=('k',),
                                            kwargs=dict(inline=True),
                                            colors=['black'],
                                            add_labels=True)
    #ax_bottom.clabel(CS)
    
    # Remove x axis labels from the top
    #ax_top.tick_params(axis='x', bottom=False, labelbottom=False)
    
    
    # Plot the bottom half
    im=da.sel(z_t=slice(depth,None)).plot.contourf('time','z_t', vmin=mn, vmax=mx,
                                                add_colorbar=False, 
                                                levels=np.linspace(mn, mx, N),
                                                ax=ax_bottom, cmap=scale_cor, add_labels=False)
    
    CS=da.sel(z_t=slice(depth,None)).plot.contour('time','z_t', vmin=mn, vmax=mx,
                                               add_colorbar=False, 
                                               levels = [mn*0.01,0,mx*0.01],
                                               colors=['blue','black','red'],
                                               kwargs=dict(inline=True),
                                               ax=ax_bottom, add_labels=True)
    #ax_bottom.clabel(CS)
    plt.grid(color='gray', linestyle='-', linewidth=0.7)

    return im, mx, mn

In [ ]:
# Set the display

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[22, 10])

gs = matplotlib.gridspec.GridSpec(2, 3, hspace=0.03, wspace=0.45, height_ratios=(2, 4))

# Display the plots

### Temperature
mn=ds_TEMP.south_atl_TEMP.mean('member_id').sel(time=slice('1850-01-01','2100-12-31')).min()
mx=ds_TEMP.south_atl_TEMP.mean('member_id').sel(time=slice('1850-01-01','2100-12-31')).max()
if np.abs(mn)>np.abs(mx):
    mx=-mn
else: 
    mn=-mx
st=f'im1, mx1, mn1=split_depth_plot(ds_{var1}.south_atl_{var1}.mean(\'member_id\'), plt.subplot(gs[0,0]), plt.subplot(gs[1,0]), depth=0.15, mx=mx*0.1, mn=mn*0.1, cor=color1)'
exec(st)

### Salinity
mn=ds_SALT.south_atl_SALT.mean('member_id').sel(time=slice('1850-01-01','2100-12-31')).min()
mx=ds_SALT.south_atl_SALT.mean('member_id').sel(time=slice('1850-01-01','2100-12-31')).max()
if np.abs(mn)>np.abs(mx):
    mx=-mn
else: 
    mn=-mx
st=f'im2, mx2, mn2=split_depth_plot(ds_{var2}.south_atl_{var2}.mean(\'member_id\'), plt.subplot(gs[0,1]), plt.subplot(gs[1,1]), depth=0.15, mx=mx*10, mn=mn*10, cor=color2)'
exec(st)

### Densitiy
mn=ds_PD.south_atl_PD.mean('member_id').sel(time=slice('1850-01-01','2100-12-31')).min()
mx=ds_PD.south_atl_PD.mean('member_id').sel(time=slice('1850-01-01','2100-12-31')).max()
if np.abs(mn)>np.abs(mx):
    mx=-mn
else: 
    mn=-mx
st=f'im3, mx3, mn3=split_depth_plot(ds_{var3}.south_atl_{var3}.mean(\'member_id\'), plt.subplot(gs[0,2]), plt.subplot(gs[1,2]), depth=0.15, mx=mx.values*1, mn=mn.values*1, cor=color3)'
exec(st)

# Set plots

ax1=plt.subplot(gs[0,0]); ax4=plt.subplot(gs[1,0]); ax2=plt.subplot(gs[0,1]); ax5=plt.subplot(gs[1,1]); ax3=plt.subplot(gs[0,2]); ax6=plt.subplot(gs[1,2])

ax1.grid(color='gray', linestyle='-', linewidth=0.7); ax2.grid(color='gray', linestyle='-', linewidth=0.7); ax3.grid(color='gray', linestyle='-', linewidth=0.7)

# Set Colorbar

axins = inset_axes(ax4,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 0, 1, 1.52),
                   bbox_transform=ax4.transAxes,
                   borderpad=0,
                   )
orig_map=plt.cm.get_cmap(color1); scale_cor=orig_map.reversed(); ticks = np.round(np.linspace(mn1, mx1, 10, endpoint=True),3)
cbar=fig.colorbar(im1,cax=axins, ticks=ticks, cmap=scale_cor, label=label1)
cbar.set_label(label=label1, fontsize=20); cbar.set_ticklabels(ticklabels=ticks,fontsize=20)

axins = inset_axes(ax5, width="5%", height="100%", loc='lower left', bbox_to_anchor=(1.05, 0, 1, 1.52), bbox_transform=ax5.transAxes, borderpad=0)
orig_map=plt.cm.get_cmap(color2); scale_cor=orig_map.reversed(); ticks = np.round(np.linspace(mn2, mx2, 10, endpoint=True),2)
cbar=fig.colorbar(im2,cax=axins, ticks=ticks, cmap=scale_cor, label=label2)
cbar.set_label(label=label2, fontsize=20); cbar.set_ticklabels(ticklabels=ticks,fontsize=20)

axins = inset_axes(ax6, width="5%", height="100%", loc='lower left', bbox_to_anchor=(1.05, 0, 1, 1.52), bbox_transform=ax6.transAxes, borderpad=0)
orig_map=plt.cm.get_cmap(color3); scale_cor=orig_map.reversed(); ticks = np.round(np.linspace(mn3, mx3, 10, endpoint=True),4); 
cbar=fig.colorbar(im3,cax=axins, ticks=ticks, cmap=scale_cor, label=label3)
cbar.set_label(label=label3, fontsize=20); cbar.set_ticklabels(ticklabels=ticks,fontsize=20)


# Set axis labels

ax4.set_xlabel('Time [Years]',fontsize=20), ax5.set_xlabel('Time [Years]',fontsize=20); ax6.set_xlabel('Time [Years]',fontsize=20)

ax1.set_ylabel('Depth [km]',fontsize=20), ax4.set_ylabel('Depth [km]',fontsize=20)

# set labels

ax4.set_title(None); ax5.set_title(None); ax6.set_title(None)

ax1.xaxis.set_ticklabels([]); ax2.xaxis.set_ticklabels([]); ax3.xaxis.set_ticklabels([])

ax3.set_yticklabels([]); ax2.set_yticklabels([]); ax5.set_yticklabels([]); ax6.set_yticklabels([])

ax5.set(ylabel=None); ax6.set(ylabel=None)

ax2.set(ylabel=None); ax3.set(ylabel=None)

ax1.invert_yaxis(); ax2.invert_yaxis(); ax3.invert_yaxis(); ax4.invert_yaxis(); ax5.invert_yaxis(); ax6.invert_yaxis() 

ax1.tick_params(axis='y', labelsize=20); ax4.tick_params(axis='y', labelsize=20)

ax2.tick_params(axis='y', labelsize=20); ax3.tick_params(axis='y', labelsize=20)

ax4.tick_params(axis='x', labelsize=20); ax5.tick_params(axis='x', labelsize=20); ax6.tick_params(axis='x', labelsize=20)

# set titles

#ax1.set_title('South Atlantic'); ax2.set_title('Western side'); ax3.set_title('Eastern side') 

ax1.set_title(None); ax2.set_title(None); ax3.set_title(None) 


at = AnchoredText("A", prop=dict(size=20), frameon=True, loc='lower left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax4.add_artist(at)

at = AnchoredText("B", prop=dict(size=20), frameon=True, loc='lower left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax5.add_artist(at)

at = AnchoredText("C", prop=dict(size=20), frameon=True, loc='lower left'); at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax6.add_artist(at)

# save

st=f'plt.savefig(\'Hovmoller_Diagram.png\',dpi=300,bbox_inches=\'tight\')'
exec(st)
plt.show()